In [1]:
import pandas as pd
import duckdb
import csv

In [2]:
PATH_TO_PARQUET_FILE = "/Users/allisoncasasola/reglab/entity-rot/data/city_ordinances_token_filtered.parquet"

In [3]:
conn = duckdb.connect()

In [4]:
# Explore the schema
schema_df = conn.execute(f"DESCRIBE SELECT * FROM '{PATH_TO_PARQUET_FILE}'").df()
print(schema_df)


           column_name column_type null   key default extra
0               source     VARCHAR  YES  None    None  None
1        source_format     VARCHAR  YES  None    None  None
2            city_slug     VARCHAR  YES  None    None  None
3    jurisdiction_name     VARCHAR  YES  None    None  None
4                state     VARCHAR  YES  None    None  None
5           state_code     VARCHAR  YES  None    None  None
6          source_file     VARCHAR  YES  None    None  None
7              node_id     VARCHAR  YES  None    None  None
8      section_heading     VARCHAR  YES  None    None  None
9       hierarchy_path     VARCHAR  YES  None    None  None
10             content     VARCHAR  YES  None    None  None
11            tiktoken      BIGINT  YES  None    None  None
12  word_tokens_approx      BIGINT  YES  None    None  None


In [5]:
num_rows = conn.execute(
    f"SELECT COUNT(*) as total FROM '{PATH_TO_PARQUET_FILE}'"
).fetchone()[0]
print(f"Total rows (AKA sections) in dataset: {num_rows:,}")

Total rows (AKA sections) in dataset: 9,509,662


In [ ]:
# Create a query to get the number of words total in the dataset (by counting the number of spaces in content)
num_words = conn.execute(
    f"SELECT SUM(LENGTH(content) - LENGTH(REPLACE(content, ' ', '')) + 1) as total_words FROM '{PATH_TO_PARQUET_FILE}'"
).fetchone()[0]
print(f"Total words in dataset: {num_words:,}")

Total words in dataset: 3,100,183,869


In [7]:
# Sample 5 random rows and display them
sample_df = conn.execute(
    f"SELECT * FROM '{PATH_TO_PARQUET_FILE}' USING SAMPLE 5"
).df()

# Display all columns without truncation
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)

sample_df

,source,source_format,city_slug,jurisdiction_name,state,state_code,source_file,node_id,section_heading,hierarchy_path,content,tiktoken,word_tokens_approx
0,alp,html,adamscountyin,Adams County,Indiana,in,adamscountyin/adamscounty_in.html,adamscountyin_934,§ 34.073 LIMITATION OF AUTHORITY.,CHAPTER 34: FINANCE AND REVENUE; TAXATION > PURCHASING AND PROCUREMENT POLICY > § 34.073 LIMITATION OF AUTHORITY.,"§ 34.073 LIMITATION OF AUTHORITY.\n\nThe limited purchasing agents (to include employees granted by their department head the authority to make purchases) shall have authority to purchase supplies and services on behalf of the county, subject to final approval and allowance of the claim by the County Board of Commissioners. Said purchases shall be subject to the threshold limitations imposed by statute and established herein. Purchases made or contracted for in contravention of the procedures outlined herein shall be made at the risk of the limited purchasing agent.\n\n(Ord. 2023-10, passed 4-11-2023)",121,147
1,alp,html,aftonmn,Afton,Minnesota,mn,aftonmn/afton_mn.html,aftonmn_1235,§ 110.04 NUDITY ON THE PREMISES OF LICENSED ESTABLISHMENTS PROHIBITED.,CHAPTER 110: ALCOHOLIC BEVERAGES > GENERAL PROVISIONS > § 110.04 NUDITY ON THE PREMISES OF LICENSED ESTABLISHMENTS PROHIBITED.,"§ 110.04 NUDITY ON THE PREMISES OF LICENSED ESTABLISHMENTS PROHIBITED.\n\n(A) (1) The City Council finds that it is in the best interests of the public health, safety and general welfare of the people of the city that nudity is prohibited as provided in this section on the premises of any establishment licensed under this chapter. This is to protect and assist the owners, operators and employees of the establishment, as well as patrons and the public in general, from harm stemming from the physical immediacy and combination of alcohol, nudity and sex.\n\n(2) The City Council especially intends to prevent any subliminal endorsement of sexual harassment or activities likely to lead to the possibility of various criminal conduct, including prostitution, sexual assault and disorderly conduct. The City Council also finds that the prohibition of nudity on the premises of any establishment licensed under this chapter, as set forth in this section, reflects the prevailing community standards of the city.\n\n(B) It is unlawful for any licensee to permit or allow any person or persons on the licensed premises when the person does not have his or her buttocks, anus, breasts and genitals covered with a nontransparent material. It is unlawful for any person to be on the licensed premises when the person does not have his or her buttocks, anus, breasts and genitals covered with a non-transparent material.\n\n(C) A violation of this section is a misdemeanor punishable as provided by law, and is justification for revocation or suspension of any liquor, wine or 3.2% malt liquor license or the imposition of a civil penalty.\n\n(Prior Code, § 4-34) Penalty, see §\n110.99",342,438
2,alp,html,alma,Alma,Kansas,ks,alma/alma_ks.html,alma_2647,14-506. SAME; EQUIPMENT ON ALL-TERRAIN VEHICLES FOR PASSENGERS.,CHAPTER XIV. TRAFFIC > ARTICLE 5. MICRO-UTILITY TRUCKS AND ALL-TERRAIN VEHICLES > 14-506. SAME; EQUIPMENT ON ALL-TERRAIN VEHICLES FOR PASSENGERS.,14-506. SAME; EQUIPMENT ON ALL-TERRAIN VEHICLES FOR PASSENGERS.\n\nAny all-terrain vehicle carrying a passenger shall be equipped with a seat and footrests for such passenger. (Ord. 452; Code 2015),55,46
3,alp,html,eastlake,Eastlake,Ohio,oh,eastlake/eastlake_oh.html,eastlake_17456,1167.41 NATURAL GRADE.,PART ELEVEN - PLANNING AND ZONING CODE > TITLE ELEVEN - Zoning Administration > CHAPTER 1167 Definitions > 1167.41 NATURAL GRADE.,"1167.41 NATURAL GRADE.\n\n""Natural grade"" means the elevation of the undisturbed natural surface of the ground adjoining the building. (Ord. 1996-029. Passed 2-12-96.)",47,36
4,alp,html,edgarwi,Edgar,Wisconsin,wi,edgarwi/edgar_wi.html,edgarwi_13526,Sec. 13-3-32 Certificates of Compliance.,Chapter 3 Shoreland

In [13]:
# List all city slugs in the dataset
city_slugs = conn.execute(
    f"SELECT DISTINCT city_slug FROM '{PATH_TO_PARQUET_FILE}'"
).df()

pd.set_option('display.max_columns', None)

print(city_slugs.to_string())

                                                         city_slug
0                                                  amberleyvillage
1                                                            argos
2                                                          argenta
3                                                         alhambra
4                                                      ashevillenc
5                                                       bainbridge
6                                                          ashland
7                                                    barboursville
8                                                        bataviail
9                                                   beavercountyut
10                                                   bedfordparkil
11                                                     battlecreek
12                                                       biglakemn
13                                                  bonnercoun

In [19]:
conn.execute(
    f"SELECT DISTINCT city_slug, jurisdiction_name, state_code FROM '{PATH_TO_PARQUET_FILE}' WHERE city_slug = 'city_of_santa_clara'"
).df()

,city_slug,jurisdiction_name,state_code
0,city_of_santa_clara,City Of Santa Clara,ca


In [27]:
# Find content that contains "The City Council may then spread the charges against the property under that statute and other pertinent statutes for certification to the County Auditor and collection along with current taxes the following year or in annual installments, not exceeding 10, as the City Council may determine in each case."ArithmeticError

data = conn.execute(
    f"SELECT * FROM '{PATH_TO_PARQUET_FILE}' WHERE content LIKE '%The City Council may then spread the charges against the property under that statute and other pertinent statutes for certification to the County Auditor and collection along with current taxes the following year or in annual installments, not exceeding 10, as the City Council may determine in each case.%'"
).df()

data


,source,source_format,city_slug,jurisdiction_name,state,state_code,source_file,node_id,section_heading,hierarchy_path,content,tiktoken,word_tokens_approx
0,alp,html,centercity,Center City,Minnesota,mn,centercity/centercity_mn.html,centercity_1413,§ 92.24 RECOVERY OF COST.,CHAPTER 92: HEALTH AND SAFETY; NUISANCES > NUISANCES > § 92.24 RECOVERY OF COST.,"§ 92.24 RECOVERY OF COST.\n\n(A)\nPersonal liability.\nThe owner of premises on which a nuisance has been abated by the city shall be personally liable for the cost to the city of the abatement, including administrative costs. As soon as the work has been completed and the cost determined, the City Clerk/Treasurer or other official shall prepare a bill for the cost and mail it to the owner. Thereupon the amount shall be immediately due and payable at the office of the City Clerk/Treasurer.\n\n(B)\nAssessment.\nAfter notice and hearing as provided in M.S. § 429.061, as it may be amended from time to time, if the nuisance is a public health or safety hazard on private property, the accumulation of snow and ice on public sidewalks, the growth of weeds on private property or outside the traveled portion of streets, or unsound or insect-infected trees, the City Clerk/Treasurer shall, on or before September 1 next following abatement of the nuisance, list the total unpaid charges along with all other charges as well as other charges for current services to be assessed under M.S. § 429.101 against each separate lot or parcel to which the charges are attributable. The City Council may then spread the charges against the property under that statute and other pertinent statutes for certification to the County Auditor and collection along with current taxes the following year or in annual installments, not exceeding 10, as the City Council may determine in each case.",304,396
1,alp,html,cologne,Cologne,Minnesota,mn,cologne/cologne_mn.html,cologne_2109,§ 95.24 RECOVERY OF COST.,CHAPTER 95: HEALTH AND SAFETY; NUISANCES > NUISANCES > § 95.24 RECOVERY OF COST.,"§ 95.24 RECOVERY OF COST.\n\n(A)\nPersonal liability.\nThe owner of premises on which a nuisance has been abated by the city shall be personally liable for the cost to the city of the abatement, including administrative costs. As soon as the work has been completed and the cost determined, the City Clerk/Treasurer or other official shall prepare a bill for the cost and mail it to the owner. Thereupon, the amount shall be immediately due and payable at the office of the City Clerk/Treasurer.\n\n(B)\nAssessment.\nAfter notice and hearing as provided in M.S. § 429.061, as it may be amended from time to time, if the nuisance is a public health or safety hazard on private property, the accumulation of snow and ice on public sidewalks, the growth of weeds on private property or outside the traveled portion of streets, or unsound or insect-infected trees, the City Clerk/Treasurer shall, on or before October 1 next following abatement of the nuisance, list the total unpaid charges along with all other charges as well as other charges for current services to be assessed under M.S. § 429.101, as it may be amended from time to time, against each separate lot or parcel to which the charges are attributable. The City Council may then spread the charges against the property under that statute and other pertinent statutes for certification to the County Auditor and collection along with current taxes the following year or in annual installments, not exceeding 10, as the City Council may determine in each case.\n\nPenalty, see §\n10.99",325,417
2,alp,html,jacksonmn,Jackson,Minnesota,mn,jacksonmn/jackson_mn.html,jacksonmn_2204,§ 94.10 RECOVERY OF COST.,CHAPTER 94: NUISANCES > GENERAL PROVISIONS > § 94.10 RECOVERY OF COST.,"§ 94.10 RECOVERY OF COST.\n\n(A)\nPersonal liability.\nThe owner of premises on which a nuisance has been abated by the city shall be personally liable for the cost to the city of the abatement, including administrative costs. As soon

In [23]:
# Generate a random sample of 10 rows from the dataset
random_sample = conn.execute(
    f"SELECT * FROM '{PATH_TO_PARQUET_FILE}' USING SAMPLE 10"

).df()

random_sample


source source_format      city_slug jurisdiction_name           state  \
0    alp          html          akron             Akron        Colorado   
1    alp          html          alamo             Alamo           Texas   
2    alp          html       adelanto          Adelanto      California   
3    alp          html      altoonaia           Altoona            Iowa   
4    alp          html    amsterdamoh         Amsterdam            Ohio   
5    alp          html    albuquerque       Albuquerque      New Mexico   
6    alp          html       eastlake          Eastlake            Ohio   
7    alp          html  eastcleveland    East Cleveland            Ohio   
8    alp          html  elizabethcity    Elizabeth City  North Carolina   
9    alp          html           edon              Edon            Ohio   

  state_code                          source_file             node_id  \
0         co               akron_co/akron_co.html          akron_1766   
1         tx                  alamo/alamo_tx.html          alamo_4466   
2         ca            adelanto/adelanto_ca.html       adelanto_3469   
3         ia            altoonaia/altoona_ia.html      altoonaia_4381   
4         oh        amsterdamoh/amsterdam_oh.html    amsterdamoh_4102   
5         nm      albuquerque/albuquerque_nm.html    albuquerque_2611   
6         oh            eastlake/eastlake_oh.html      eastlake_13610   
7         oh  eastcleveland/eastcleveland_oh.html  eastcleveland_2712   
8         nc      elizabethcity/elizabeth_nc.html  elizabethcity_4792   
9         oh                    edon/edon_oh.html           edon_1304   

                                  section_heading  \
0                  9-3-5: CONSTRUCTION OF SEWERS:   
1                        9-2-19: JUDICIAL REVIEW:   
2                5.68.060 Signs and Solicitation.   
3                     135.11 FAILURE TO MAINTAIN.   
4                    § 75.28 PERMITTED OPERATION.   
5                     § 2-19-3 APPROVAL OF GRANT.   
6                   906.09 PERFORMANCE STANDARDS.   
7                          § 192.036 INDIVIDUALS.   
8                   § 157.06 INSPECTION OF SITES.   
9  185.082 REFUNDABLE CREDIT FOR QUALIFYING LOSS.   

                                                                                                                                                                  hierarchy_path  \
0                                                                                                                               ARTICLE B. CATS > 9-3-5: CONSTRUCTION OF SEWERS:   
1                                                                                                                     CHAPTER 2 SUBSTANDARD BUILDINGS > 9-2-19: JUDICIAL REVIEW:   
2                                                                                                            PART 6. VIOLATIONS; SEVERABILITY > 5.68.060 Signs and Solicitation.   
3                                                                                                                                SUPPLEMENT RECORD > 135.11 FAILURE TO MAINTAIN.   
4                       CHAPTER 75: BICYCLES, MOTORCYCLES, AND OFF-ROAD VEHICLES > SNOWMOBILES, OFF-HIGHWAY MOTORCYCLES, AND ALL-PURPOSE VEHICLES > § 75.28 PERMITTED OPERATION.   
5                                                   CHAPTER 2: GOVERNMENT > ARTICLE 19: COMMUNITY ENERGY EFFICIENCY DEVELOPMENT (CEED) BLOCK GRANT > § 2-19-3 APPROVAL OF GRANT.   
6  PART NINE - STREETS, UTILITIES AND PUBLIC SERVICES CODE > TITLE ONE - Street and Sidewalk Areas > CHAPTER 906 Soil Sediment Pollution Control > 906.09 PERFORMANCE STANDARDS.   
7                                                                                                                                     IMPOSITION OF TAX > § 192.036 INDIVIDUALS.   
8                                                                                                                     STORM WATER ADVISORY BOARD > § 157.06 INSPECTION O